# Regrid Genna's CESM MLD output for use by TE (regrid to JRA mesh)
# This was run on NCAR's derecho where all the model output is stored.

In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import xesmf as xesmf
import pandas as pd
from netCDF4 import Dataset, MFDataset
import os

In [2]:
#---
# pick a random JRA file to get lat/lon info
#---

lr_grid = xr.open_dataset('/glade/p/cesmdata/cseg/inputdata/ocn/jra55/v1.5_noleap/JRA.v1.5.v_10.TL319.2021.220505.nc',\
                          decode_times=False)['v_10'].isel(time=0)
lr_grid = lr_grid.drop_vars('time')
lon_jra = lr_grid['longitude'].values
lat_jra = lr_grid['latitude'].values
lr_grid

<xarray.DataArray 'v_10' (latitude: 320, longitude: 640)> Size: 819kB
[204800 values with dtype=float32]
Coordinates:
  * longitude  (longitude) float64 5kB 0.0 0.5625 1.125 ... 358.3 358.9 359.4
  * latitude   (latitude) float64 3kB -89.57 -89.01 -88.45 ... 88.45 89.01 89.57
Attributes:
    note:           10m reference height data
    standard_name:  northward_wind
    long_name:      Northward Near-Surface Wind
    comment:        Northward component of the near surface wind
    units:          m s-1
    cell_methods:   area: mean time: point
    cell_measures:  area: areacella
    history:        2022-04-30T19:30:42Z altered by CMOR: Treated scalar dime...

In [3]:
#---
# load Genna's data
#---

path1 = '/glade/derecho/scratch/gclow/archive/g.e22.GOMIPECOIAF_JRA.TL319_g17_sst2/ocn/hist/'
file_chl = 'g.e22.GOMIPECOIAF_JRA.TL319_g17_sst2.pop.h.nday1.1980-01-01.nc'

var = 'HMXL_DR_2' #'XMXL_2'
ff = xr.open_dataset(path1+file_chl)[var]#.isel(z_t=0)
ff = ff.rename({'TLONG': 'longitude','TLAT': 'latitude'})
ff = ff.drop_vars('ULONG')
ff = ff.drop_vars('ULAT')
ff = ff.drop_vars('time')
#ff

In [4]:
regridder_jra = xesmf.Regridder(ff, lr_grid, 'bilinear') 

In [5]:
ds_rg = regridder_jra(ff)

In [6]:
#ds_rg.to_netcdf('/glade/work/cnissen/CESM_chl_regridded/test.nc')
#ds_rg
data_to_save = np.copy(ds_rg.values)
print(data_to_save.shape)

(365, 320, 640)


In [7]:
#-----
# loop over all years and save as netcdf file 
#-----

savepath = '/glade/work/cnissen/CESM_MLD_2_regridded/'
save_netcdf = True

path1 = '/glade/derecho/scratch/gclow/archive/g.e22.GOMIPECOIAF_JRA.TL319_g17_sst2/ocn/hist/'
var   = 'MLD' # name in regridded data
var1  = 'HMXL_DR_2' #'XMXL_2' # name in CESM
var_description = 'density-based MLD (variable '+var1+' in original CESM output)'

fv = -999

year_list = np.arange(1978,2018+1,1) #1979,2018+1,1)

for yy in range(0,len(year_list)):

    file_chl = 'g.e22.GOMIPECOIAF_JRA.TL319_g17_sst2.pop.h.nday1.'+str(year_list[yy])+'-01-01.nc'

    # PFT1
    ff = xr.open_dataset(path1+file_chl)[var1]  #.isel(z_t=0)
    ff = ff.rename({'TLONG': 'longitude','TLAT': 'latitude'})
    ff = ff.drop_vars('ULONG')
    ff = ff.drop_vars('ULAT')
    ff = ff.drop_vars('time')
    # create mapping file
    regridder_jra = xesmf.Regridder(ff, lr_grid, 'bilinear') 
    # regrid field
    ds_rg = regridder_jra(ff)
    # create numpy array
    data_to_save = np.copy(ds_rg.values)
    # close file
    ff.close()

    if save_netcdf:
        netcdf_name = var+'_JRA_grid_'+str(year_list[yy])+'-01-01.nc'
        if not os.path.exists(savepath+netcdf_name):
            print('Create file '+savepath+netcdf_name)
            w_nc_fid = Dataset(savepath+netcdf_name, 'w', format='NETCDF4_CLASSIC')
            w_nc_fid.contact = 'Cara Nissen, cara.nissen@colorado.edu'
            w_nc_fid.source_file = path1+file_chl
            w_nc_fid.script      = '/glade/u/home/cnissen/scripts/regrid_CESM_MLD_output_JRA_mesh.ipynb'
            # create dimension & variable
            w_nc_fid.createDimension('time', data_to_save.shape[0]) 
            w_nc_fid.createDimension('latitude', data_to_save.shape[1]) 
            w_nc_fid.createDimension('longitude', data_to_save.shape[2]) 
            w_nc_var1 = w_nc_fid.createVariable(var, 'f4',('time','latitude','longitude'),fill_value=fv)
            w_nc_var1.description = var_description
            w_nc_var1.units = 'm'
            w_nc_var1 = w_nc_fid.createVariable('latitude', 'f4',('latitude'),fill_value=fv)
            w_nc_var1.description = 'Latitude'
            w_nc_var1.units = 'deg N'
            w_nc_var1 = w_nc_fid.createVariable('longitude', 'f4',('longitude'),fill_value=fv)
            w_nc_var1.description = 'Longitude (0:360)'
            w_nc_var1.units = 'deg E'
            w_nc_fid.close()

        data_to_save[np.isnan(data_to_save)] = fv
        data_to_save[data_to_save==0] = fv

        # get rid of white line!
        data_to_save[:,:,568] = data_to_save[:,:,567]
        data_to_save[:,:,569] = data_to_save[:,:,570]

        w_nc_fid = Dataset(savepath+netcdf_name, 'r+', format='NETCDF4_CLASSIC') 
        w_nc_fid.variables[var][:,:] = data_to_save
        w_nc_fid.variables['latitude'][:] = lat_jra
        w_nc_fid.variables['longitude'][:] = lon_jra
        w_nc_fid.close()  

        print('Succesfully saved year '+str(year_list[yy]))

print('done')


Create file /glade/work/cnissen/CESM_MLD_2_regridded/MLD_JRA_grid_1978-01-01.nc
Succesfully saved year 1978
Create file /glade/work/cnissen/CESM_MLD_2_regridded/MLD_JRA_grid_1979-01-01.nc
Succesfully saved year 1979
Create file /glade/work/cnissen/CESM_MLD_2_regridded/MLD_JRA_grid_1980-01-01.nc
Succesfully saved year 1980
Create file /glade/work/cnissen/CESM_MLD_2_regridded/MLD_JRA_grid_1981-01-01.nc
Succesfully saved year 1981
Create file /glade/work/cnissen/CESM_MLD_2_regridded/MLD_JRA_grid_1982-01-01.nc
Succesfully saved year 1982
Create file /glade/work/cnissen/CESM_MLD_2_regridded/MLD_JRA_grid_1983-01-01.nc
Succesfully saved year 1983
Create file /glade/work/cnissen/CESM_MLD_2_regridded/MLD_JRA_grid_1984-01-01.nc
Succesfully saved year 1984
Create file /glade/work/cnissen/CESM_MLD_2_regridded/MLD_JRA_grid_1985-01-01.nc
Succesfully saved year 1985
Create file /glade/work/cnissen/CESM_MLD_2_regridded/MLD_JRA_grid_1986-01-01.nc
Succesfully saved year 1986
Create file /glade/work/cnis